<h1 style="align: center;font-size: 18pt;"> Cineromycin B</h1>

<hr style="height:2.5px">

Summary: In our previous work (DOI: [10.1002/jcc.23738](https://onlinelibrary-wiley-com.libproxy.temple.edu/doi/pdf/10.1002/jcc.23738")), we determine solution-state conformational populations of the 14-membered macrocycle cineromycin B, using a combination of previously published sparse Nuclear Magnetic Resonance (NMR) observables and replica exchange molecular dynamic/Quantum mehcanics (QM)-refined conformational ensembles. Cineromycin B is a 14-membered macrolide antibiotic that has become increasingly known for having activity against methicillin-resistant Staphylococcus Aureus (MRSA). In this example, we show how to calculate the consistency of computational modeling with experiment, and the relative importance of reference potentials and other model parameters. 

<hr style="height:2.5px">

Supplemental Information: [the workflow](https://biceps.readthedocs.io/en/latest/workflow.html), [more examples](https://biceps.readthedocs.io/en/latest/examples/index.html)
First, we need to `import biceps`. More information of these python objects can be found .

Please feel free to use this example as a template.

In [ ]:
import sys, os, glob, cPickle
import numpy as np
import biceps

Now we need to specify input files and output files folder.

In [ ]:
#########################################
# Lets' create input files for BICePs
############ Preparation ################
# Specify necessary argument values
path='NOE/*txt'                # precomputed distances for each state
states=100                     # number of states
indices='atom_indice_noe.txt'  # atom indices of each distance
exp_data='noe_distance.txt'    # experimental NOE data 
top='cineromycinB_pdbs/0.fixed.pdb'    # topology file 
dataFiles = 'noe_J'            # output directory of BICePs formated input file from this scripts
out_dir=dataFiles               

#p=Preparation('noe',states=states,indices=indices,
#    exp_data=exp_data,top=top,data_dir=data_dir)  # the type of data needs to be specified {'noe', 'J', 'cs_H', etc}
#p.write(out_dir=out_dir)      # raw data will be converted to a BICePs readable format to the folder specified

In [ ]:
#########################################
# Let's create our ensemble of structures
############ Initialization #############    
# Specify necessary argument values
data = biceps.sort_data(dataFiles)   # sorting data in the folder and figure out what types of data are used
print data
print len(data),len(data[0])

energies_filename =  'cineromycinB_QMenergies.dat'
energies = np.loadtxt(energies_filename)*627.509  # convert from hartrees to kcal/mol
energies = energies/0.5959   # convert to reduced free energies F = f/kT
energies -= energies.min()  # set ground state to zero, just in case
outdir = 'results_ref_normal'

# Make a new directory if we have to
if not os.path.exists(outdir):
    os.mkdir(outdir)

Next, we need to specify number of steps for BICePs sampling. We recommend to run at least 1M steps for converged Monte Carlo samplings. In BICePs, we also prepare functions of checking convergence of Monte Carlo. More information can be found [here].

In [ ]:
nsteps = 1000000 # number of steps of MCMC simulation

We need to specify what lambda values we want to use in BICePs samplings. Briefly, lambda values are similar to the parameters used in free energy perturbation (FEP) and has effect on the BICePs score. The lambda values represent how much prior information from computational modeling is included in BICePs sampling (1.0 means all, 0.0 means none). As we explained in [this work](https://pubs.acs.org/doi/10.1021/acs.jpcb.7b11871), one can consider BICePs score as the relative free energy change between different models. More lambda values will increase the samplings for [multistate Bennett acceptance ratio (MBAR)](http://www.alchemistry.org/wiki/Multistate_Bennett_Acceptance_Ratio) predictions in free energy change and populations. However more lambda values also will slow down the whole process of BICePs (as more samplings need to run), so balancing the accuracy and efficiency is important. To successfully finish a BICePs sampling, lambda values of 0.0 and 1.0 are necessary. Based on our experience, the lambda values of 0.0,0.5,1.0 are good enough for BICePs sampling.

Print what experimental observables are included in BICePs sampling. This is optional and provides a chance for double check before running BICePs sampling.

In this tutorial, we used both J couplings and NOE (pairwise distances) in BICePs sampling. 

In [ ]:
lambda_values = [0.0,0.5,1.0]
res = biceps.list_res(data)
print res

Another key parameter for BICePs set-up is the type of reference potential for each experimental observables. More information of reference potential can be found [here](https://biceps.readthedocs.io/en/latest/theory.html).

Three reference potentials are supported in BICePs: uniform ('uniform'), exponential ('exp'), Gaussian ('gau').  

As we found in previous research, exponential reference potential is useful in most cases. Some higher level task may require more in reference potential selection (e.g [force field parametrization](https://pubs.acs.org/doi/10.1021/acs.jpcb.7b11871)).

(Noet: It will be helpful to print out what is the order of experimental observables included in BICePs sampling as shown above.)

In [ ]:
ref=['uniform','exp']
uncern=[[0.05,20.0,1.02],[0.05,5.0,1.02]]
gamma = [0.2,5.0,1.01]

If you want, you can specify nuisance parameters (uncertainty in experiments) range but it's not required. Our default parameters range is broad enough.
"gamma" is a scaling factor converting distances to NOE intensity in experiments. It is only used when NOE restraint is ued in BICePs sampling.

Now we need to set up BICePs samplings and feed arguments using variables we defined above. In most cases, you don't need to modify this part as long as you defined all these parameters shown above. 

In [ ]:
for j in lambda_values:
    verbose = False # True
    lam = j
    # We will instantiate a number of Structure() objects to construct the ensemble
    ensemble = []
    for i in range(energies.shape[0]):
        if verbose:
            print '\n#### STRUCTURE %d ####'%i
        ensemble.append([])
        for k in range(len(data[0])):
            File = data[i][k]
            if verbose:
                print File
            R = biceps.init_res('cineromycinB_pdbs/0.fixed.pdb',
                              lam,energies[i],File,ref[k],uncern[k],gamma)
            ensemble[-1].append(R)
    print ensemble

    ##########################################
    # Next, let's do some posterior sampling
    ########## Posterior Sampling ############
    
    sampler = biceps.PosteriorSampler(ensemble)
    sampler.compile_nuisance_parameters()

    sampler.sample(nsteps)  # number of steps

    print 'Processing trajectory...',
    print 'Writing results...',
    sampler.traj.process_results(os.path.join(outdir,'traj_lambda%2.2f.npz'%lam))  # compute averages, etc.
    print '...Done.'
    
    sampler.traj.read_results(os.path.join(outdir,'traj_lambda%2.2f.npz'%lam))

    print 'Pickling the sampler object ...',
    outfilename = 'sampler_lambda%2.2f.pkl'%lam
    print outfilename,
    fout = open(os.path.join(outdir, outfilename), 'wb')
    # Pickle dictionary using protocol 0.
    cPickle.dump(sampler, fout)
    fout.close()
    print '...Done.'

Then, we need to run functions in [Analysis](https://biceps.readthedocs.io/en/latest/api.html#analysis) to get predicted populations of each conformational states and BICePs score. A figure of posterior distribution of populations and nuisance parameters will be plotted as well. 

In [ ]:
#########################################
# Let's do analysis using MBAR and plot figures
############ MBAR and Figures ###########
# Specify necessary argument values
A = biceps.Analysis(states, outdir) 
A.plot()

The output files include: population information ("populations.dat"), figure of sampled parameters distribution ("BICePs.pdf"), BICePs score information ("BS.dat").




First, let's take a look at the populations file.

In [ ]:
fin = open('populations.dat','r')
text = fin.read()
fin.close()
print text

There are 100 rows corresponding to 100 clustered states. 6 columns corresponding to populations of each state (row) for 3 lambda values (first 3 columns) and population change (last 3 columns). 

Then let's take a look at the output figure ("BICePs.pdf").




The top left panel shows the population of each states in the presence (y-axis) and absense (x-axis) of computational modeling prior information. You can find some states (e.g state 38, 59) get awarded after including computational prior information. If you check [this work](https://onlinelibrary.wiley.com/doi/full/10.1002/jcc.23738) you will see how misleading the results will be if we only use experimental restraints without computational prior information. 
The other three panels show the distribution of nuisance parameters in the presence or absence of computational modeling information. It may not be clear in this example due to the limit of finite sampling, but based on our experience, including prior information from our simulations will lower the nuisance parameters than only using experimental restraints.

<h6 style="align: justify;font-size: 12pt"># <span style="color:red;">NOTE</span>: The following cell is for pretty notebook rendering</h6>

In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../../theme.css", "r").read()
    return HTML(styles)
css_styling()